# 双层网络，最基本网络结构，批量梯度下降优化

# 导入库

In [1]:
import numpy as np
import _pickle as cPickle
import gzip
from scipy import ndimage
import matplotlib.pyplot as plt
%matplotlib inline

# 载入图像相关函数

In [4]:
def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.
    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.
    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.
    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.
    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('../data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f, encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper(augmentation=False):
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.
    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    if augmentation: training_inputs, training_results = expend_training_data(training_inputs, training_results)
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)


def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

# Augment training data
def expend_training_data(images, labels):

    expanded_images = []
    expanded_labels = []

    j = 0 # counter
    for x, y in zip(images, labels):
        j = j+1
        if j%100==0:
            print ('expanding data : %03d / %03d' % (j,np.size(images,0)))

        # register original data
        expanded_images.append(x)
        expanded_labels.append(y)

        # get a value for the background
        # zero is the expected value, but median() is used to estimate background's value
        bg_value = np.median(x) # this is regarded as background's value
        image = np.reshape(x, (-1, 28))

        for i in range(4):

            ## Augmentation via Rotation
            # rotate the image with random degree
            angle = np.random.randint(-15,15,1)
            new_img = ndimage.rotate(image,angle,reshape=False, cval=bg_value)

            # shift the image with random distance
            shift = np.random.randint(-2, 2, 2)
            new_img_ = ndimage.shift(new_img,shift, cval=bg_value)

            # register new training data
            expanded_images.append(np.reshape(new_img_, [784,1]))
            expanded_labels.append(y)


    return expanded_images, expanded_labels

# 载入数据

In [5]:
training_data, validation_data, test_data = load_data_wrapper()

# 训练参数

In [11]:
n_epoch = 30
learning_rate = 1
batch_size = 10

# 网络结构

In [12]:
n_node_input = 784
n_node_hidden = 100
n_node_output = 10

# 权重与偏置

In [13]:
W2=np.random.randn(n_node_hidden, n_node_input)
b2=np.random.randn(n_node_hidden, 1)

W3=np.random.randn(n_node_output, n_node_hidden)
b3=np.random.randn(n_node_output, 1)

# 激活函数

In [14]:
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

# 开始训练

In [15]:
test_errors = []
training_errors = []
n = len(training_data)

for j in range(n_epoch):

    ## Stochastic Gradient Descent
    np.random.shuffle(training_data)

    # for each batch
    sum_of_training_error = 0
    for k in range(0, n, batch_size):
        batch = training_data[k:k+batch_size]

        # average gradient for samples in a batch
        sum_gradient_b3 = 0
        sum_gradient_b2 = 0
        sum_gradient_W3 = 0
        sum_gradient_W2 = 0

        # for each sample
        for x, y in batch:
            ## Feed forward

            a1 = x
            z2 = np.dot(W2, a1) + b2
            a2 = sigmoid(z2)
            z3 = np.dot(W3, a2) + b3
            a3 = sigmoid(z3)

            ## Backpropagation

            # Step 1: Error at the output layer [Quadratic Cost]
            delta_3 = (a3-y)*sigmoid_prime(z3)
            # Step 2: Error relationship between two adjacent layers
            delta_2 =  sigmoid_prime(z2)*np.dot(W3.transpose(), delta_3)
            # Step 3: Gradient of C in terms of bias
            gradient_b3 = delta_3
            gradient_b2 = delta_2
            # Step 4: Gradient of C in terms of weight
            gradient_W3 = np.dot(delta_3, a2.transpose())
            gradient_W2 = np.dot(delta_2, a1.transpose())

            # update gradients
            sum_gradient_b3 += gradient_b3
            sum_gradient_b2 += gradient_b2
            sum_gradient_W3 += gradient_W3
            sum_gradient_W2 += gradient_W2

            ## Training Error
            sum_of_training_error += int(np.argmax(a3) != np.argmax(y))


        # update weights & biases
        b3 -= learning_rate * sum_gradient_b3 / batch_size
        b2 -= learning_rate * sum_gradient_b2 / batch_size
        W3 -= learning_rate * sum_gradient_W3 / batch_size
        W2 -= learning_rate * sum_gradient_W2 / batch_size

    # Report Training Error
    print("[TRAIN_ERROR] Epoch %02d: %5d / %05d" % (j, sum_of_training_error, n))
    training_errors.append(np.float(sum_of_training_error) / n)

    ### Test
    n_test = len(test_data)
    sum_of_test_error = 0
    for x, y in test_data:
        ## Feed forward

        a1 = x
        z2 = np.dot(W2, a1) + b2
        a2 = sigmoid(z2)
        z3 = np.dot(W3, a2) + b3
        a3 = sigmoid(z3)

        ## Test Error
        # in test data, label info is a number not one-hot vector as in training data
        sum_of_test_error += int(np.argmax(a3) != y)

    # Report Test Error
    print("[ TEST_ERROR] Epoch %02d: %5d / %05d" % (j, sum_of_test_error, n_test))

    test_errors.append(np.float(sum_of_test_error)/n_test)
    
print("done!")

[TRAIN_ERROR] Epoch 00: 26006 / 50000
[ TEST_ERROR] Epoch 00:  4343 / 10000
[TRAIN_ERROR] Epoch 01: 17132 / 50000
[ TEST_ERROR] Epoch 01:  2677 / 10000
[TRAIN_ERROR] Epoch 02: 12588 / 50000
[ TEST_ERROR] Epoch 02:  2525 / 10000
[TRAIN_ERROR] Epoch 03: 12032 / 50000
[ TEST_ERROR] Epoch 03:  2470 / 10000
[TRAIN_ERROR] Epoch 04: 11708 / 50000
[ TEST_ERROR] Epoch 04:  2367 / 10000
[TRAIN_ERROR] Epoch 05:  8121 / 50000
[ TEST_ERROR] Epoch 05:  1552 / 10000
[TRAIN_ERROR] Epoch 06:  7117 / 50000
[ TEST_ERROR] Epoch 06:  1492 / 10000
[TRAIN_ERROR] Epoch 07:  6910 / 50000
[ TEST_ERROR] Epoch 07:  1464 / 10000
[TRAIN_ERROR] Epoch 08:  6734 / 50000
[ TEST_ERROR] Epoch 08:  1449 / 10000
[TRAIN_ERROR] Epoch 09:  6615 / 50000
[ TEST_ERROR] Epoch 09:  1437 / 10000
[TRAIN_ERROR] Epoch 10:  6489 / 50000
[ TEST_ERROR] Epoch 10:  1420 / 10000
[TRAIN_ERROR] Epoch 11:  6420 / 50000
[ TEST_ERROR] Epoch 11:  1387 / 10000
[TRAIN_ERROR] Epoch 12:  6323 / 50000
[ TEST_ERROR] Epoch 12:  1374 / 10000
[TRAIN_ERROR